# Implementation Ass_5

### Implement tree search 

Tree search algorithm for board games like TTT: 

The (theoretical) whole search space is 9! = 362 880 different states. But terminal nodes (winning combinations) end branches beforehand. So actual possible states is < 20 000.  --> feasible to make exhaustive searches. No limits in search tree other that hyperparameter max-iter (good for time aspect).  

- MCTS: cycle of four steps. Select, expand, explore, and backup. Monte-carlo methods perform randomly selected of actions. Ie. make sure to not leave out any exploration opportunities. (something like Kai's first version) 

- Minimax algorithm: This is a classic algorithm for turn-based games. It works by recursively exploring all possible moves and their outcomes, and then selecting the move that maximizes the minimum outcome.

- Alpha-beta pruning: This is a variant of the minimax algorithm that eliminates branches of the search tree that are guaranteed to be worse than previously explored branches. This can significantly reduce the search space and speed up the algorithm.

- 

Different design choices: 

- the roll-out policy(move selection) of your player: How does my player select moves?  

For actual moves we give input. But our move in the simulation of the program should be a move towards a possible row/diagonal. --> implement a method to find and return random "liberties" that are "aggressive moves"?   

- the policy of your opponent: how are the moves of the program selected?

Choose the highest ucb move which should be the simulated move with the most wins per visit. Assuming that the simulation has shed light on most (all) good moves and increased their UCB. 

- the selection policy in your search tree: what node is selected in the search tree? 

The simulation should cover enough of the search three to make a decision that maximizes the probability of winning (wins/visits). To ensure that no good moves are missed out due to finding good but not the best move too early selection of suboptimal moves is done randomly as part of the exploration. But before, a "minimal exploration" is done by controlling whether all nodes are fully expanded can be carried out to guarantee that all nodes are visited at least once (do with if visits =0 for all legal moves).   

- the updates to your search tree ("back-up")

For the updates, we make sure to increment the wins and visits all the way to the top. The new values will automatically adjust the ucb score over time. During simulation, the search three is expanded randomly until termination in each loop, not fully expanded nodes are progressivly expanded and when all children have been visited at least once it is again a matter of random choice.     

note: if we simply want to find the optimal node we can loop the entire search space with DFS or BFS. But that is costly and ineffective. --> hence we randomly explore large parts, make sure to get some minimal degree of exploration, and otherwise uniformly expand the different parts without any respect to the number of wins. The question remains if we could increase the speed of finding and sedimenting the best moves by designing a policy that takes the highest ucb score but where the ucb score initially is artificially increased for not visited nodes, and later is equal to the actual rate of wins/visits.   

note: innan varje move borde vår motståndare göra en granskning om detta state är ett state innan förlust. Isåfall blockerar den 

In [1]:
# imports
import numpy as np 

## Classes Node & MCTS

In [2]:
"""
Overall we need to define two classes: Node and MCTS. 
The Node class represents a node in the MCTS tree, and contains 
the state, parent node, child nodes, win count, and visit count. 

The MCTS class represents the MCTS algorithm itself, and contains the exploration constant, 
maximum number of iterations, and the methods for running the search and simulating games.
"""

class Node:
    def __init__(self, state, player):
        self.state = state
        self.player = player
        self.parent = None
        self.children = [] # list with children
        self.wins = 0 # use for V = average reward? 
        self.visits = 0 # cannot divide by zero --> adjust the UCB expression with "1+visists" --> evalaute
        #self.untried_moves = [] # pop from this... 

    


    def select_child(self, exploration_constant = np.sqrt(2)):
        #compute ucb score for all kids 
        ucb_values = [child.wins/(1+child.visits) + exploration_constant * np.sqrt(np.log(1+self.visits)/(1+child.visits)) 
        for child in self.children]
        #exploration_constant *= 0.9 # TODO adjust the constant
        #ucb_values.sort(reverse=True)
        #print("Scores for top three: ")
        #print(ucb_values[:3])
        return self.children[np.argmax(ucb_values)]

    # expand (but only if never expanded before)
    def expand(self):
        possible_moves = [(i, j) for i in range(3) for j in range(3) if self.state[i][j] == 0]
               
        # handle if there are no possible moves, ie. a tie
        if not possible_moves: #if empty return self
            #print('The list is empty, no possible moves!')
            return self

        # is the node fully explored? 
        for legal_child in possible_moves:
            if legal_child.visits == 0:
                return legal_child
         

        else:
            # TODO implement a method that correctly pops and explores
            if self.untried_moves: #if moves left untried 
                #print("Poped in expansion")
                next_move = self.untried_moves.pop()
                new_state_list = np.copy(self.state)
                new_state_list[next_move] = self.player # sets currect player on new state position
                child_node = Node(new_state_list, -self.player)
                child_node.parent = self
                self.children.append(child_node)
                return child_node

            # create children from all possible moves and return the selected best for rollout step
            for new_child_move in possible_moves:
                new_state_list = np.copy(self.state)
                new_state_list[new_child_move] = self.player # sets currect player on new state position
                child_node = Node(new_state_list, -self.player)
                child_node.parent = self
                self.children.append(child_node)
            
            child_chosen_for_simulation = self.select_child() #exploration constant default
            return child_chosen_for_simulation 
        

    """Implement better policy for opponent.
    For example, use a Minimax algorithm to choose the opponent's moves.
    If first move (middle), else if, the connecting positions"""

    def update(self, result):
        self.visits += 1
        if result == self.player:
            self.wins += 1       
        #print("###############update completed!")



In [3]:
class MCTS:
    def __init__(self, exploration_constant= np.sqrt(2), max_iterations=10):
        self.exploration_constant = exploration_constant
        self.max_iterations = max_iterations

    def search(self, initial_state):
        root_node = Node(initial_state, 1) 
        for i in range(self.max_iterations):
            node = root_node # start every search from root
            #select & expand accordingly
            if not node.children: #if no children exists
                #expand leaf for the most promissing child
                most_promising_child_node = node.expand() 
                
            else: # children exist
                existing_child_node = node.select_child(self.exploration_constant) # select child with highest UCB score for rollout        
                most_promising_child_node = existing_child_node.expand()    
            #rollout
            result = self.run_simulation(most_promising_child_node) # simulate form new child on this MCTS obj
            #backup
            while node: #stops when at root node
                node.update(result) # update wins and visits
                node = node.parent # move up the three
            #print("Simulations Nr: " + str(i))
        #act in line with search results    
        best_child = root_node.children[np.argmax([child.visits for child in root_node.children])] #action based on most visited
        action = best_child.state - root_node.state
        #return self.select_action(best_child) # promotes exploration
        print( "Best move found: \n" + str(action) + " now the bord is: ")
        return action
        #return root_node.children[np.argmax([child.visits for child in root_node.children])].state - root_node.state
    """
    def traverse(): # TODO, called once in every search, before rollout
    # while current node is not terminal node --> if not fully expanded, then expand (with untried node).  Else, choose best child node. 
    # return:  
    """
    def run_simulation(self, node): #aka rollout
        this_state = node.state
        this_player = node.player
        
        #print("New simulation with: \n" +str(this_state)) #visualize
        c=0
       
        while True:  #until terminal state reached
            #possible_moves = [(i, j) for i in range(3) for j in range(3) if state[i][j] == 0] # detected bug, it misses the certain values before in row?
            c+=1
            #return either 0 (for tie) or -1/1 (for the winning player) as a reward for this rollout 
            
            #expand and choose best action until either tie or win 
            if self.termination_control(state=this_state, current_player=this_player): #wining condition
                #print("simulation: we have a winner")
                return this_player
    
            if node is node.expand(): # tie 
                #print("Simulation: It's a tie!")
                return 0    
            # update node
            
            next_node = node.expand()
            node = next_node  # loop forever in case of a tie now. --> since expand resturn self, identify this here!
            #print(c)
            player = -player

    """
    def select_action(self, node): 
        for child in node.children:
            #Loop below is done to promote exploration and avoid always choosing the same child node
            if child.visits == node.visits:
                #action is represented as the difference between the state lists 
                #of the best child node and the state of the current node.
                return child.state - node.state 
        # return the action that selects the most visited child
        return node.children[np.argmax([child.visits for child in node.children])].state - node.state
    """
    def termination_control(self, state, current_player): #only investigates wins
        for i in range(3): # check every row and colum for "straight wins"
            if state[i][0] == state[i][1] == state[i][2] == current_player:
                return True
            if state[0][i] == state[1][i] == state[2][i] == current_player:
                return True
        if state[0][0] == state[1][1] == state[2][2] == current_player: # "diagonal wins"
            return True
        if state[0][2] == state[1][1] == state[2][0] == current_player:
            return True
        return False

## Run an example

In [4]:
# create a new MCTS instance to learn
mcts = MCTS(exploration_constant=1.0, max_iterations=100)


In [5]:
111
1
11
# initialize the game state
game_state = np.zeros((3, 3))
print("Current borad: ")
print(game_state)

# play the game
while True:
    
    
    # human player's turn
    print("###Human player's turn")
    row = int(input("Enter row: "))-1
    col = int(input("Enter col: "))-1
    game_state[row][col] = -1
    print(game_state)
    if mcts.termination_control(game_state, -1):
        print("Human player wins!")
        break
    
    # computer player's turn
    print("###Computer player's turn")
    # computer explores the search tree & takes best action found 
    copy_game_state = game_state.copy()
    action = mcts.search(copy_game_state)
    game_state += action 
    print(game_state)
    if mcts.termination_control(game_state, 1):
        print("Computer player wins!")
        break
    if not np.any(game_state == 0):
        print("It's a tie!")
        break


Current borad: 
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
###Human player's turn


KeyboardInterrupt: Interrupted by user

## Experimentation with exploration constant

Try different exploration constants and maximum iterations to see how they affect the performance of the algorithm.

# Evaluation

Evaluate your algorithm and comment on its pros and cons. For example, is it fast? Is it sample efficient? Is the learned policy competitive? Does it lose? Would you, as a human, beat it? Would it scale well to larger grids such as 4x4 or 5x5?

# Next gen TTT AI

We could try using a neural network to estimate the value of each state instead of simulating games to the end. This could speed up the search and improve the performance of the algorithm.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1262dda2-abb7-4af7-a1b6-72164064af5a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>